# Shopee Mini Project

## Libaray and Data loading

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
np.random.seed(0)

In [2]:
#import train and predict data
data = pd.read_csv("D:/Shopee/reactivation_data/training.csv")
test_data = pd.read_csv("D:/Shopee/reactivation_data/predict.csv")

In [3]:
#import other information
transactions = pd.read_csv("D:/Shopee/reactivation_data/transactions_MY.csv")
users = pd.read_csv("D:/Shopee/reactivation_data/user_profiles_MY.csv")
likes =pd.read_csv("D:/Shopee/reactivation_data/likes.csv")
voucher_date = pd.read_csv("D:/Shopee/reactivation_data/voucher_distribution_active_date.csv")
voucher = pd.read_csv("D:/Shopee/reactivation_data/voucher_mechanics.csv")

In [4]:
#import behavior data
log=[]
for i in range(31):
    log.append(pd.read_csv("D:/Shopee/reactivation_data/view_log_" +str(i) + ".csv"))
    

In [5]:
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(1487231880))
int(time.mktime(time.strptime('2015-09-02 14:38:38', '%Y-%m-%d %H:%M:%S')))

1441175918

## Data Cleaning

First we dealing with the target. We could found that if **repurchase_x** is 1, then for any $y>x$ we have **repurchase_y is** 1.

In [6]:
used_y =data["used?"]
repurchase_y = data[["repurchase_15?","repurchase_30?","repurchase_60?","repurchase_90?"]]
X = data[["userid","promotionid_received","voucher_code_received","voucher_received_time"]]

In [7]:
tmp_df = repurchase_y["repurchase_90?"] >= repurchase_y["repurchase_60?"]
print(tmp_df.all())
tmp_df = repurchase_y["repurchase_60?"] >= repurchase_y["repurchase_30?"]
print(tmp_df.all())
tmp_df = repurchase_y["repurchase_30?"] >= repurchase_y["repurchase_15?"]
print(tmp_df.all())

True
True
True


In [8]:
repurchase_reform_y = repurchase_y.sum(1)

However, there is no necessity to repurchase to used to coupon at least

In [9]:
tmp_df = ((used_y == 0) & (repurchase_reform_y > 0)) 
tmp_df.any()

True

Then we uniform the date format for further using.

In [12]:
def epoch2datetime(t):
    tt = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(t))
    return pd.to_datetime(tt)

In [13]:
X.loc[:,"voucher_received_time"] = X["voucher_received_time"].apply(epoch2datetime)
test_data.loc[:,"voucher_received_time"] = test_data["voucher_received_time"].apply(epoch2datetime)
transactions.loc[:,"order_time"] = transactions["order_time"].apply(epoch2datetime)
voucher_date.loc[:,"voucher_received_time"] = voucher_date["voucher_received_time"].apply(epoch2datetime)
likes.loc[:,"ctime"] = likes["ctime"].apply(epoch2datetime)
users.loc[:,"registration_time"] =pd.to_datetime(users["registration_time"])
users.loc[:,"birthday"] =pd.to_datetime(users["birthday"])


TypeError: an integer is required (got type Timestamp)

Now we deal with the transaction data. We should consider which kind of the transaction data we could use. Fot classify the *used?*, the order happened after the voucher received date should not be consider as the featurs. We would calculate the number of orders and average price.

In [ ]:
def extract_transaction_information(row):
    userid = userid_t['userid']
    t = userid_t["voucher_received_time"]
    tmp = transactions.loc[ transactions['userid']==userid]
    tmp = tmp.loc[tmp['order_time'] < t]
    tmp = tmp["total_price"]
    num = tmp.shape[0]
    avg = tmp.sum()/num if num>0 else 0
    return [num,avg]

#extract_transaction_information(X.loc[20020,["userid","voucher_received_time"]])

trans_extract = X[["userid","voucher_received_time"]].apply(extract_transaction_information,axis=1)


In [ ]:
#trans_extract = pd.DataFrame(columns = ['num_orders', 'average_price'])
#trans_extract.head()
#for i in range(X.shape[0]):
#    tmp = extract_transaction_information(X.loc[i,"userid"], X.loc[i,"voucher_received_time"])
#    trans_extract.loc[i] = tmp
    


Last we deal with the **likes** data. We could count how many items that the users like and unlike befor the voucher was received.

In [ ]:
def extract_likes_information(row):
    userid = userid_t['userid']
    t = userid_t["voucher_received_time"]
    tmp = likes.loc[ likes['userid']==userid]
    tmp = tmp.loc[tmp['ctime'] < t]
    tmp = tmp["status"]
    num_likes = tmp.sum()
    num_unlikes = tmp.shape[0] - num_likes
    return [num_likes ,num_unlikes]
likes_extract = X[["userid","voucher_received_time"]].apply(extract_likes_information,axis=1)

In [84]:
transactions['promotionid_used']=transactions['promotionid_used'].apply(int)
transactions.loc[ transactions['userid']==80157 ]

,orderid,userid,shopid,total_price,order_time,voucher_code_used,promotionid_used
0,125381,80157,12978,1100000,2015-07-09 14:28:41,NaN,NaN
37328,49759879,80157,914698,8600000,2016-09-28 21:29:26,NaN,NaN
50112,212514634,80157,24941925,3572000,2017-08-22 23:02:03,SHOPEE20RE10W34,194882.0
452213,25515531,80157,1476563,1390000,2016-06-21 15:29:49,NaN,NaN
831052,239873,80157,12978,1500000,2015-08-12 21:52:40,NaN,NaN
885400,49535102,80157,914698,7500000,2016-09-28 04:51:40,NaN,NaN
1016995,165071050,80157,16176252,3804000,2017-07-02 12:09:06,SHOPEE20RE10W27,137171.0
1021241,239927,80157,12978,2000000,2015-08-12 22:01:15,NaN,NaN
1021421,2655570,80157,28646,1900000,2015-12-14 22:23:26,1212WOMEN,7241.0
1176818,239957,80157,12978,2000000,2015-08-12 22:07:51,NaN,NaN


## Features Construction

First we append the information and merge to our origin **X**

In [10]:
# append user information
X = pd.merge(X, users,  how='left', left_on=['userid'],right_on=['userid'])
# append voucher information
X = pd.merge(X, voucher,  how='left', left_on=['promotionid_received'],right_on=['promotionid_received'])
# append the voucher active_x
X = pd.merge(X, voucher_date,  how='left', left_on=['userid','promotionid_received','voucher_code_received','voucher_received_time'], right_on = ['userid','promotionid_received','voucher_code_received','voucher_received_time'])
del X['voucher_received_date']


# append transaction information

# append likes information



In [26]:
X[["userid","voucher_received_time"]].head()

,userid,voucher_received_time
0,13014,2017-08-09 00:37:11
1,13046,2017-04-20 19:26:04
2,13109,2017-08-09 01:20:38
3,22837,2017-07-05 18:18:17
4,28948,2017-03-22 12:08:35


In [79]:
X.loc[ X['userid']==80157 ]

,userid,promotionid_received,voucher_code_received,voucher_received_time,active_0,active_1,active_2,active_3,active_4,active_5,...,active_29,active_30,discount,max_value,phone_verified,registration_time,is_seller,email_verified,gender,birthday
14074,80157,57394,REV20BA3,2017-02-03 10:58:10,2.0,4.0,2.0,1.0,4.0,3.0,...,3.0,5.0,20,1000000,1,2015-07-02 15:13:45,0,1,2.0,1982-06-03
24806,80157,188866,SHOPEE20RE10W33,2017-08-09 01:21:34,18.0,2.0,4.0,4.0,2.0,4.0,...,10.0,2.0,20,1000000,1,2015-07-02 15:13:45,0,1,2.0,1982-06-03
195118,80157,105724,REV50B43,2017-04-20 19:05:18,6.0,6.0,14.0,8.0,4.0,2.0,...,3.0,6.0,50,1500000,1,2015-07-02 15:13:45,0,1,2.0,1982-06-03
451477,80157,58115,REV20BA4,2017-02-16 16:01:18,6.0,5.0,11.0,10.0,15.0,4.0,...,2.0,4.0,20,1000000,1,2015-07-02 15:13:45,0,1,2.0,1982-06-03
504450,80157,154106,SHOPEE20RE10W28,2017-07-05 19:15:09,8.0,5.0,10.0,24.0,18.0,6.0,...,2.0,5.0,20,1000000,1,2015-07-02 15:13:45,0,1,2.0,1982-06-03
507988,80157,137171,SHOPEE20RE10W27,2017-06-30 14:58:36,6.0,NaN,NaN,2.0,NaN,NaN,...,6.0,NaN,20,1000000,1,2015-07-02 15:13:45,0,1,2.0,1982-06-03
525840,80157,100120,REV50AB6,2017-02-24 18:02:38,1.0,1.0,2.0,6.0,6.0,3.0,...,5.0,7.0,50,1500000,1,2015-07-02 15:13:45,0,1,2.0,1982-06-03
593347,80157,116717,REV50B45,2017-05-15 12:44:51,12.0,4.0,6.0,6.0,6.0,6.0,...,4.0,2.0,50,1500000,1,2015-07-02 15:13:45,0,1,2.0,1982-06-03
635893,80157,105021,REV50B41,2017-04-11 20:32:15,2.0,2.0,2.0,2.0,6.0,2.0,...,2.0,NaN,50,1500000,1,2015-07-02 15:13:45,0,1,2.0,1982-06-03
